昨日は、lightGBMに「築年数」「面積」「間取り」「契約期間」「その部屋のある階数」「全体の階数」「所在地」「最寄り駅」「最寄り駅までの所要時間」をいれてみて、過学習に陥った。<br>
本日は正則化を試みる。

In [139]:
import pandas as pd
import numpy as np
import lightgbm as lgb

訓練データの読み込み

In [140]:
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange_scores = pd.read_csv('room_arrange_scores.csv')
contract_span = pd.read_csv('contract_span.csv')
floor_scores = pd.read_csv('floor_scores.csv')
Floor_scores = pd.read_csv('capital_floor_scores.csv')
wards = pd.read_csv('wards.csv')
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')

rent = pd.read_csv('rent.csv')

テストデータの読み込み

In [141]:
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange_scores = pd.read_csv('test_room_arrange_scores.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_wards = pd.read_csv('test_wards.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')

訓練データの作成

In [142]:
X_train = pd.concat([house_age, area_size, room_arrange_scores, contract_span, floor_scores, Floor_scores, wards, stations, minits], axis=1)
y_train = rent

In [143]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

テストデータの作成

In [144]:
X_test = pd.concat([test_house_age, test_area_size, test_room_arrange_scores, test_contract_span, test_floor_scores, test_Floor_scores, test_wards, test_stations, test_minits], axis=1)

lightGBMで正則化を行う。

In [145]:
X_train

,築年数,面積,間取り得点,契約期間,所在階,全体の階数,市区町村,最寄り駅,所要時間
1382,589,28.35,4,3.0,2.0,12.0,13,99,4.0
15190,29,25.45,2,2.0,3.0,3.0,3,417,8.0
28196,365,44.55,4,2.0,7.0,11.0,21,86,13.0
18762,4,33.91,3,2.0,7.0,13.0,3,16,4.0
13923,143,22.62,2,2.0,2.0,10.0,15,385,2.0
...,...,...,...,...,...,...,...,...,...
13123,181,50.01,4,2.0,1.0,2.0,19,61,NaN
19648,0,25.80,2,2.0,2.0,4.0,4,141,6.0
9845,388,36.72,3,2.0,NaN,NaN,20,51,NaN
10799,374,19.00,1,0.0,1.0,2.0,4,154,13.0


In [146]:
X_test

,築年数,面積,間取り得点,契約期間,所在階,全体の階数,市区町村,最寄り駅,所要時間
0,588,50.22,5,0.0,8.0,8.0,1,1,15.0
1,2,20.88,1,2.0,3.0,4.0,2,2,6.0
2,280,26.93,2,2.0,1.0,4.0,3,3,10.0
3,434,23.57,2,2.0,1.0,2.0,4,4,10.0
4,544,50.00,5,2.0,4.0,4.0,4,4,12.0
...,...,...,...,...,...,...,...,...,...
31257,1,25.66,2,2.0,6.0,8.0,3,154,3.0
31258,186,22.71,1,0.0,8.0,15.0,22,389,2.0
31259,12,45.76,4,2.0,10.0,14.0,19,175,10.0
31260,184,55.20,2,0.0,14.0,14.0,22,396,3.0


In [147]:
category_lists = ['市区町村','最寄り駅']

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse'
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['市区町村', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
c:

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001346 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1089
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 9
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35666.5	valid_1's rmse: 40678.1
[20]	training's rmse: 22792.8	valid_1's rmse: 30744.4
[30]	training's rmse: 18244.7	valid_1's rmse: 27457.7
[40]	training's rmse: 16313.8	valid_1's rmse: 26121.2
[50]	training's rmse: 15124.1	valid_1's rmse: 25296.8
[60]	training's rmse: 14285.3	valid_1's rmse: 24719.4
[70]	training's rmse: 13634.6	valid_1's rmse: 24307.2
[80]	training's rmse: 13096.2	valid_1's rmse: 24029.2
[90]	training's rmse: 12623.5	valid_1's rmse: 23792
[100]	training's rmse: 12203	valid_1's rmse: 23589
[110]	training's rmse: 11833.7	valid_1's rmse: 23428.9


In [148]:
test_stations

,最寄り駅
0,1
1,2
2,3
3,4
4,4
...,...
31257,154
31258,389
31259,175
31260,396
